In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import shutil

# 모듈 임포트
from utils import mean_absolute_percentage_error
from dataset import ReviewDataset
from models import ASRec
from train_eval import train_model, evaluate_model, recommend_topk_for_all_users


# 파일 경로
data_file_path = 'review_business_5up_5aspect_3sentiment_vectorized_clean.json'
df = pd.read_json(data_file_path, lines=True)

# 필요한 컬럼 추출
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

# sentiment_vector_dim을 동적으로 결정
sentiment_vector_dim = len(df_processed['sentiment_vector'].iloc[0]) if not df_processed.empty else 32

# 데이터 분할 (70:10:20 학습:검증:테스트 비율)
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8 # 전체 데이터의 10% (학습 데이터의 1/8)
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")
print(f"감성 벡터 차원: {sentiment_vector_dim}")


# GPU 사용

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# dataset 및 DataLoader 생성

train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df)


# 모델 파라미터 및 초기화

# 최적 파라미터
best_params = {
    'aspect_mlp_hidden_dims': [64, 32],
    'batch_size': 128,
    'embedding_dim': 64,
    'final_mlp_hidden_dims': [32, 16],
    'learning_rate': 0.001,
    'user_biz_mlp_hidden_dims': [128, 64]
}

print("\n" + "="*50)
print(f"최적 파라미터 적용 {best_params}")
print("="*50)

final_embedding_dim = best_params['embedding_dim']
final_learning_rate = best_params['learning_rate']
final_batch_size = best_params['batch_size']
final_user_biz_mlp_dims = best_params['user_biz_mlp_hidden_dims']
final_aspect_mlp_dims = best_params['aspect_mlp_hidden_dims']
final_final_mlp_dims = best_params['final_mlp_hidden_dims']

final_model = ASRec(num_users, num_businesses, final_embedding_dim,
                    final_user_biz_mlp_dims, final_aspect_mlp_dims, final_final_mlp_dims,
                    sentiment_vector_dim).to(device)

final_criterion = nn.MSELoss()
final_optimizer = optim.Adam(final_model.parameters(), lr=final_learning_rate)

final_train_loader = DataLoader(train_dataset, batch_size=final_batch_size, shuffle=True)
final_val_loader = DataLoader(val_dataset, batch_size=final_batch_size, shuffle=False)
final_test_loader = DataLoader(test_dataset, batch_size=final_batch_size, shuffle=False)

## 모델 학습

final_epochs = 50
final_patience = 10
final_min_delta = 0.0005
final_model_path = 'best_as_rec_model.pt'

train_model(
    model=final_model,
    train_loader=final_train_loader,
    val_loader=final_val_loader,
    criterion=final_criterion,
    optimizer=final_optimizer,
    epochs=final_epochs,
    patience=final_patience,
    min_delta=final_min_delta,
    model_path=final_model_path,
    device=device
)


# 모델 평가

evaluation_metrics = evaluate_model(
    model=final_model,
    test_loader=final_test_loader,
    device=device,
    model_path=final_model_path
)


전체 데이터 수: 451185
학습 데이터 수: 315829 (70.00%)
검증 데이터 수: 45119 (10.00%)
테스트 데이터 수: 90237 (20.00%)
감성 벡터 차원: 15
사용 장치: cuda

최적 파라미터 적용 {'aspect_mlp_hidden_dims': [64, 32], 'batch_size': 128, 'embedding_dim': 64, 'final_mlp_hidden_dims': [32, 16], 'learning_rate': 0.001, 'user_biz_mlp_hidden_dims': [128, 64]}

 모델 학습 시작
에포크 1/50, 학습 손실: 0.7319, 검증 RMSE: 0.7083
RMSE 개선됨. 모델 저장: 0.7083
에포크 2/50, 학습 손실: 0.4765, 검증 RMSE: 0.6908
RMSE 개선됨. 모델 저장: 0.6908
에포크 3/50, 학습 손실: 0.4392, 검증 RMSE: 0.6865
RMSE 개선됨. 모델 저장: 0.6865
에포크 4/50, 학습 손실: 0.4085, 검증 RMSE: 0.6841
RMSE 개선됨. 모델 저장: 0.6841
에포크 5/50, 학습 손실: 0.3814, 검증 RMSE: 0.6896
RMSE 개선되지 않음. (1/10)
에포크 6/50, 학습 손실: 0.3558, 검증 RMSE: 0.7082
RMSE 개선되지 않음. (2/10)
에포크 7/50, 학습 손실: 0.3292, 검증 RMSE: 0.7048
RMSE 개선되지 않음. (3/10)
에포크 8/50, 학습 손실: 0.3036, 검증 RMSE: 0.7171
RMSE 개선되지 않음. (4/10)
에포크 9/50, 학습 손실: 0.2762, 검증 RMSE: 0.7249
RMSE 개선되지 않음. (5/10)
에포크 10/50, 학습 손실: 0.2508, 검증 RMSE: 0.7415
RMSE 개선되지 않음. (6/10)
에포크 11/50, 학습 손실: 0.2259, 검증 RMSE: 0.7490
RMSE 개선되